In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import transformers
from tqdm.auto import tqdm, trange

from openpyxl.reader.excel import load_workbook
from openpyxl.styles import Side, Border, NamedStyle, Alignment

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
model_name = "codellama/CodeLlama-7b-hf"

In [3]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

In [348]:
del model
torch.cuda.empty_cache()

In [ ]:
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name
)
for param in model.parameters():
    param.requires_grad=False
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

config.json:   0%|          | 0.00/637 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [356]:
import peft

peft_config = peft.PromptTuningConfig(
    peft_type="PROMPT_TUNING",
    task_type="SEQ_2_SEQ_LM",
    num_virtual_tokens=30
)
model = peft.get_peft_model(model, peft_config)
model.print_trainable_parameters()
model.to(device)

trainable params: 92,160 || all params: 783,242,240 || trainable%: 0.0118


PeftModelForSeq2SeqLM(
  (base_model): PeftModelForSeq2SeqLM(
    (base_model): T5ForConditionalGeneration(
      (shared): Embedding(32128, 1024)
      (encoder): T5Stack(
        (embed_tokens): Embedding(32128, 1024)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): Linear(in_features=1024, out_features=1024, bias=False)
                  (k): Linear(in_features=1024, out_features=1024, bias=False)
                  (v): Linear(in_features=1024, out_features=1024, bias=False)
                  (o): Linear(in_features=1024, out_features=1024, bias=False)
                  (relative_attention_bias): Embedding(32, 16)
                )
                (layer_norm): T5LayerNorm()
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (1): T5LayerFF(
                (DenseReluDense): T5DenseGatedActDense(
       

In [357]:
def get_i_prompt_info(sheet, i):
    partner = str(sheet[f'E{i}'].value)
    desc = str(sheet[f'U{i}'].value)
    return partner + ' ' + desc

In [352]:
def get_i_out_info(sheet, i):
    return str(sheet[f'X{i + 6}'].value)

In [353]:
def get_data():
    prompts = []
    outs = []
    for i in range(1, 3):
        book_prompt = load_workbook(f'input_{i}.xlsx')
        sheet_prompt = book_prompt.worksheets[0]
        book_out = load_workbook(f'out_{i}.xlsx')
        sheet_out = book_out.worksheets[1]
        for i, row in enumerate(sheet_prompt.rows):
            prompts.append(get_i_prompt_info(sheet_prompt, i + 1))
            outs.append(get_i_out_info(sheet_out, i + 1))
    return prompts, outs

In [329]:
prompts, outs = get_data()

print(len(prompts))
print(len(outs))

425
425


In [354]:
print(prompts[8])
print(outs[8])

40701810420100300024
9723132301
ООО МКК "БРИДЖ КАПИТАЛ" Оплата по ДЗ № 15 от 02.08.2022 г. (перечисление денежных средств третьему лицу по поручению ООО Дилижанс Транс по Заявка № 000119782 от 19.12.2022 г Без налога (НДС)
услуги по перевозкам от ООО МКК "БРИДЖ КАПИТАЛ"


In [355]:
assert None not in prompts
assert None not in outs

In [358]:
from transformers import get_linear_schedule_with_warmup

lr = 3e-2
num_epochs = 3

optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(prompts) * num_epochs),
)

In [359]:
#model = model.to(device)

while loss.item() > 0.1:
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(prompts)):
        inputs = tokenizer(prompts[step], text_target=outs[step], return_tensors="pt").to(device)
        outputs = model(**inputs)

        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        if loss.item() < 0.1:
            break
    print(loss.item())

  0%|          | 0/425 [00:00<?, ?it/s]

2.268209218978882


  0%|          | 0/425 [00:00<?, ?it/s]

1.9817605018615723


  0%|          | 0/425 [00:00<?, ?it/s]

...

2.07997989654541


  0%|          | 0/425 [00:00<?, ?it/s]

2.147122383117676


  0%|          | 0/425 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [360]:
model.print_trainable_parameters()

trainable params: 92,160 || all params: 783,242,240 || trainable%: 0.0118


In [361]:
print(loss.item())

1.9122734069824219


In [365]:
prompt = prompts[3]
print(prompt)
#model.to(device)
model.eval()
inputs = tokenizer(
    prompt, return_tensors="pt").to(device)

print(inputs)

outputs = model.generate(input_ids=inputs.input_ids)
print(outputs)
print("\nOutput:", tokenizer.decode(outputs[0]))
print(prompt)

40802810568000027512
181802149628
Индивидуальный предприниматель МЕРЗЛЯКОВ АНДРЕЙ АЛЕКСАНДРОВИЧ Оплата по договору АН2650, счет № 1014 от 23.12.2022г. за транспортные услуги. НДС не облагается.
{'input_ids': tensor([[ 1283,  2079,  2577, 12869,  3651, 19568, 25988,  2122,   507, 20829,
         27357,  4314,  2577,     3,     2,  7184,  5814,  2795, 25610,  5814,
          3700, 22141,     2,  7184,     2,     3,     2, 30051,     2, 14709,
         14391, 20447, 14982,  6588,     2,     3,     2,     3,     2,     3,
             2,     3,     2, 17148, 15517,     3,     2,  2044,     3,  5814,
          2044,     2, 16811,  2044, 23912,     3,     2,  2688,  1752,     6,
         12681,     2, 15042,   465,   335,  2534,     3, 17059,  1902,     5,
          9368,  1755,  2884,     2,     5,     3,     2,  2533,     3,  6725,
          7948,  7184,  5345,     2, 18632,  6725,  7184,     2,  1757,     3,
          3700,  5345,  6588,  3700,     2,  2795,     5,     3,     2,     3,
  

ValueError: input_ids must be provided for Peft model generation